In [62]:
import win32gui
import win32api
import numpy as np
from PIL import ImageGrab
import cv2
import pandas as pd

In [63]:
# Get the window of minesweeper

class_name = "TMain" # class of the window

title_name = "Minesweeper Arbiter" # title of the window

hwnd = win32gui.FindWindow(class_name, title_name)

# print(hwnd)

if hwnd:
    left, top, right, bottom = win32gui.GetWindowRect(hwnd)
    # print(str(left)+" "+str(top)+" "+str(right)+" "+str(bottom))
else:
    print("Can't find minesweeper")

100 205 386 605


In [64]:
# Adjust the rectangle

left += 44
top += 178
right += 78
bottom += 98

rect = (left, top, right, bottom)

img = ImageGrab.grab().crop(rect)

img.show()

In [65]:
block_width, block_height = 20, 20

blocks_x = int((right-left)/block_width)
blocks_y = int((bottom-top)/block_height)

In [66]:
def SegmentBlock(img, x, y):
    x1, y1 = x*block_height, y*block_width
    x2, y2 = x1+block_height, y1+block_width
    return img.crop((y1, x1, y2, x2))

blocks = [[0 for i in range(blocks_x)] for i in range(blocks_y)]

for x in range(blocks_x):
    for y in range(blocks_y):
        blocks[x][y] = SegmentBlock(img, x, y)

In [67]:
def exist_rgb(rgb_array, img_rgb):
    for i in range(len(img_rgb)):
        for element in img_rgb[i]:
            if np.array_equal(rgb_array, element):
                return True
    else:
        return False

In [68]:
from ctypes import sizeof


def identify_block(blocks, blocks_y, blocks_x):
    category_block = []
    category_block.append(np.array([192,192,192])) # unclicked
    category_block.append(np.array([255,0,0])) # 1
    category_block.append(np.array([0,128,0])) # 2
    category_block.append(np.array([0,0,255])) # 3
    category_block.append(np.array([128,0,0])) # 4
    category_block.append(np.array([0,0,128])) # 5
    category_block.append(np.array([128,128,0])) # 6
    category_block.append(np.array([0,0,0])) # 7
    category_block.append(np.array([114,114,114])) # 8
    category_block.append(np.array([255,255,255])) # white
    #  print(len(category_block))
    # 9: flagged
    # -1: unclicked
    # 0: blank
    # -2: mine
    mine_block = [[0 for i in range(blocks_x)] for i in range(blocks_y)]
    for x in range(blocks_x):
        for y in range(blocks_y):
            block = blocks[x][y]
            img_rgb = cv2.cvtColor(np.array(block), cv2.COLOR_BGR2RGB)
            if exist_rgb(category_block[1], img_rgb):
                mine_block[x][y] = 1
            elif exist_rgb(category_block[2], img_rgb):
                mine_block[x][y] = 2
            elif exist_rgb(category_block[3], img_rgb):
                if not exist_rgb(category_block[7], img_rgb):
                    mine_block[x][y] = 3
                elif np.array_equal(np.array([0,0,255]), img_rgb[3,4]):
                    mine_block[x][y] = -2
                else:
                    mine_block[x][y] = 9
            elif exist_rgb(category_block[4], img_rgb):
                mine_block[x][y] = 4
            elif exist_rgb(category_block[5], img_rgb):
                mine_block[x][y] = 5
            elif exist_rgb(category_block[6], img_rgb):
                mine_block[x][y] = 6
            elif exist_rgb(category_block[7], img_rgb):
                if exist_rgb(category_block[9], img_rgb):
                        mine_block[x][y] = -2
                else:
                    mine_block[x][y] = 7
            elif exist_rgb(category_block[8], img_rgb):
                mine_block[x][y] = 8
            elif exist_rgb(category_block[9], img_rgb):
                mine_block[x][y] = -1
            else:
                mine_block[x][y] = 0
    return mine_block

In [69]:
current = identify_block(blocks, blocks_y, blocks_x)

In [70]:
df = pd.DataFrame(current)
print("Current minesweeper board")
print(df)

Current minesweeper board
    0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15
0   -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
1   -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
2   -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
3    1   1   2  -1  -1  -1  -1  -1  -1  -1   2   1   1   1  -1  -1
4    0   0   1  -1  -1  -1   2  -1  -1  -1   1   0   0   1  -1  -1
5    0   0   1   1   1   1  -1  -1  -1  -1   1   0   0   1  -1  -1
6    0   0   0   0   0   1   1   3  -1  -1   1   1   0   1  -1  -1
7    0   0   0   0   0   0   0   2  -1  -1  -1   1   0   1  -1  -1
8    1   1   1   0   0   0   0   1   1  -1  -1   1   0   2  -1  -1
9   -1  -1   3   2   2   1   1   0   1   2  -1   2   1   2  -1  -1
10  -1  -1  -1  -1  -1  -1   2   1   3  -1  -1  -1  -1  -1  -1  -1
11  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
12  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
13  -1  -1  -1  -1  -1  -1  -1  -1  

In [71]:
current = np.array(current)
marked = np.copy(current)
while True:
    temp = np.copy(marked)
    for i in range(1, len(current[0])-1):
        for j in range(1, len(current)-1):
            block = marked[j-1:j+2, i-1:i+2]
            center = marked[j][i]
            count_unclicked = np.count_nonzero(block == -1)
            count_flagged = np.count_nonzero(block == 9)
            if (count_unclicked+count_flagged) == center:
                block[block==-1] = 9
            elif count_flagged == center:
                block[block==-1] = 0
            marked[j-1:j+2, i-1:i+2] = block
    if np.array_equal(temp, marked):
        break
df_marked = pd.DataFrame(marked)
print("Marked with possible mines")
print(df_marked)


Marked with possible mines
    0   1   2   3   4   5   6   7   8   9   10  11  12  13  14  15
0    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
1    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
2    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
3    1   1   2   0   0   0   0   0   0   0   2   1   1   1   0   0
4    0   0   1   9   0   0   2   0   0   0   1   0   0   1   9  -1
5    0   0   1   1   1   1   0   0   0   0   1   0   0   1   0  -1
6    0   0   0   0   0   1   1   3   0   0   1   1   0   1   0  -1
7    0   0   0   0   0   0   0   2   9   0   9   1   0   1   9  -1
8    1   1   1   0   0   0   0   1   1   0   0   1   0   2   0  -1
9    0   9   3   2   2   1   1   0   1   2   0   2   1   2   9  -1
10  -1   0   9   9   0   9   2   1   3   9   9  -1  -1  -1  -1  -1
11  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
12  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1  -1
13  -1  -1  -1  -1  -1  -1  -1  -1 